In [1]:
!pip install tweepy pandas scikit-learn nltk


In [5]:

import tweepy
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAADnNxQEAAAAAZKcNIj9AbqPlXTMJq%2BBwyY0Dqxc%3DJXMTTcQRGspFg8V4Z2p22OgJjwTV5d9ZLVhL3WCzjF3Qc80XL2"  # Replace with your Bearer Token

client = tweepy.Client(bearer_token=BEARER_TOKEN)


def fetch_tweets(query, max_results=10):
    try:
        tweets = client.search_recent_tweets(query=query, max_results=max_results, tweet_fields=["text"])
        tweet_data = [tweet.text for tweet in tweets.data] if tweets.data else []
        return tweet_data
    except tweepy.TweepyException as e:
        print(f"Error fetching tweets: {e}")
        return []


nltk.download('stopwords')
nltk.download('punkt')

def preprocess_text(text):
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text.lower())
    tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
    return ' '.join(tokens)


data = {
    "text": [
        "Vaccines are harmful and cause autism",
        "The Earth is not flat; it's a sphere",
        "COVID-19 is caused by 5G networks",
        "Climate change is real and caused by human activities"
    ],
    "label": [1, 0, 1, 0]
}
df = pd.DataFrame(data)


df["processed_text"] = df["text"].apply(preprocess_text)


X = df["processed_text"]
y = df["label"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('classifier', MultinomialNB())
])

pipeline.fit(X_train, y_train)


y_pred = pipeline.predict(X_test)
print("Model Accuracy:", accuracy_score(y_test, y_pred))


def detect_misinformation(tweets):
    processed_tweets = [preprocess_text(tweet) for tweet in tweets]
    predictions = pipeline.predict(processed_tweets)
    results = [{"tweet": tweet, "is_misinformation": bool(pred)} for tweet, pred in zip(tweets, predictions)]
    return results


query = "misinformation lang:en"
tweets = fetch_tweets(query=query, max_results=10)

if tweets:
    results = detect_misinformation(tweets)
    print("\nReal-Time Misinformation Detection Results:")
    for result in results:
        status = "Misinformation" if result["is_misinformation"] else "True Information"
        print(f"- Tweet: {result['tweet']}\n  Status: {status}\n")
else:
    print("No tweets fetched or available for analysis.")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Model Accuracy: 0.0

Real-Time Misinformation Detection Results:
- Tweet: RT @ZaleskiLuke: Elon Musk and X Are the Top Misinformation Spreaders Online https://t.co/ii81uz7AbB
  Status: Misinformation

- Tweet: A video circulating on SM falsely claimed to show a Hindu temple being vandalized in Bangladesh. In reality, the footage dates back to 2021 mob attack on a temple in Pak, spreading misinformation that fuels hatred &amp; worsens religious tensions in Bangladesh
https://t.co/9F2wflY9XW
  Status: Misinformation

- Tweet: Vaccine misinformation distorts science – a biochemist explains how RFK Jr. and his lawyer’s claims threaten public health https://t.co/vAnQ4roqDm via @ConversationUS
  Status: Misinformation

- Tweet: RT @Jonathan_Elk: As an Israeli-Lebanese Christian, I’m outraged by the lies coming from the President of Ireland about my country, Israel.…
  Status: Misinformation

- Tweet: RT @VinceGottalotta: @ZWV134 @CivilLost @KenWali1 @Tomgrif399 @KiltedRef @JeffJun80346519 @8